### Import required Libraries

In [39]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [168]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics

### Collect text from this goodreads.com website , so you have to need of Internet connection

In [169]:
url = 'http://www.goodreads.com/quotes/tag/{}?page={}'

### In which Type of Quotes , So here 'Love' And 'Inspirations' Taken

In [180]:
emotions = ['love','religion']

### Then formating the url Because whenever visit This website More than Pages and  Quotes are available so Here love quotes and page-3 is taken  From As Per this

In [181]:
complete = url.format(emotions[0],3)

In [182]:
complete

'http://www.goodreads.com/quotes/tag/love?page=3'

### Here, using Beautiful Then We have Question In Mind What is Beautiful And what is used for???

### Beautiful Soup is a Python library for pulling data out of HTML and XML files.




In [183]:
def get_quotes(complete):
    data = requests.get(complete)
    soup = BeautifulSoup(data.text)
    divs = soup.find_all('div',attrs={'class':'quoteText'})
    quotes = [div.text.strip().split('\n')[0][1:-1] for div in divs]
    return quotes

#### Here We only pull Data in Div tag And named on Class is quotetext. Because take only those data We need. 

#### Removed All Whitespaces and Unnecessary words  By using Strip() split in this Function.

In [184]:
quotes  = get_quotes(complete)

### In which Checks How many Love And inspiration Quotes In page

In [207]:
X,y  = [], []
for emotion in emotions:
    for i in range(1,10):
        complete = url.format(emotion,i)
        quotes = get_quotes(complete)
        X.extend(quotes)
        y.extend([emotion]*len(quotes))       #[love]*3 = [love,love,love]
        print(f"Processed Page {i} for {emotion}")

Processed Page 1 for love
Processed Page 2 for love
Processed Page 3 for love
Processed Page 4 for love
Processed Page 5 for love
Processed Page 6 for love
Processed Page 7 for love
Processed Page 8 for love
Processed Page 9 for love
Processed Page 1 for religion
Processed Page 2 for religion
Processed Page 3 for religion
Processed Page 4 for religion
Processed Page 5 for religion
Processed Page 6 for religion
Processed Page 7 for religion
Processed Page 8 for religion
Processed Page 9 for religion


### Size Of Quotes

In [186]:
len(X) , len(y)

(420, 420)

### Now, This Data Saved in csv File...

In [187]:
df = pd.DataFrame(list(zip(y,X)),columns=['emotion','quotes'])

In [188]:
df.to_csv('emotion.csv',index=False)

In [189]:
df.head()

,emotion,quotes
0,love,"I'm selfish, impatient and a little insecure. ..."
1,love,You've gotta dance like there's nobody watchin...
2,love,You know you're in love when you can't fall as...
3,love,A friend is someone who knows all about you an...
4,love,Darkness cannot drive out darkness: only light...


### Buid CountVectorizer Model

In [190]:
vect = CountVectorizer(max_features=500)

In [191]:
vect.fit(X)

CountVectorizer(max_features=500)

In [192]:
len(vect.vocabulary_)

500

In [193]:
X_mod = vect.transform(X).todense()

In [194]:
len(X_mod)

420

In [195]:
tokenizer = RegexpTokenizer('\w+')
sw = set(stopwords.words('english'))
ps = PorterStemmer()

### Apply tokenization And  Remove Stopwords

In [196]:
def getStemmedQuote(quote):
    quote = quote.lower()
    
    #tokenize
    tokens = tokenizer.tokenize(quote)
    
    #remove stopwords
    new_tokens = [token for token in tokens if token not in sw]
    
    stemmed_token = [ps.stem(token) for token in new_tokens]
    
    clean_quote = ' '.join(stemmed_token)
    
    return clean_quote

def getStemmedQuotes(quotes):
    d = []
    for quote in quotes:
        d.append(getStemmedQuote(quote))
    return d    

In [197]:
X = getStemmedQuotes(X)
X

['selfish impati littl insecur make mistak control time hard handl handl worst sure hell deserv best',
 'gotta danc like nobodi watch love like never hurt sing like nobodi listen live like heaven earth',
 'know love fall asleep realiti final better dream',
 'friend someon know still love',
 'dark cannot drive dark light hate cannot drive hate love',
 'accept love think deserv',
 'better hate love',
 'life truli believ find someon complet turn world around tell thing never share anoth soul absorb everyth say actual want hear share hope futur dream never come true goal never achiev mani disappoint life thrown someth wonder happen wait tell know share excit embarrass cri hurt laugh make fool never hurt feel make feel like good enough rather build show thing make special even beauti never pressur jealousi competit quiet calm around worri think love thing seem insignific peopl note song walk becom invalu treasur kept safe heart cherish forev memori childhood come back clear vivid like young

In [198]:
X_mod = vect.transform(X).todense()

In [199]:
X_mod.shape

(420, 500)

### Splitting Data Into Trainset and Testset

In [200]:
X_train,X_test,y_train,y_test = train_test_split(X_mod,y,test_size=0.33,random_state=200)

In [201]:
X_mod

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [202]:
model = BernoulliNB()

In [203]:
model.fit(X_train,y_train)

BernoulliNB()

In [204]:
y_pred = model.predict(X_test)

In [205]:
y_test = np.array(y_test)

### model Accuracy how often is the classifier correct?

In [206]:
print("Accuracy :",metrics.accuracy_score(y_test,y_pred))

Accuracy : 0.8129496402877698
